# ***Notebook Purpose***
*We want to create our own model to transform user data into 2 Dimensional Data using PCA. We will using tf.math, tf.linalg, etc. to scale data and count pca value from user data.*

*After we create the model, we will save it as saved_model so we can deploy it using tf-serving*

# **Import library**

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.metrics.pairwise import euclidean_distances

# **Tidying Up Datasets**

## Load Datasets

In [3]:
# Load Tour Guide Datasets
guide_df = pd.read_csv('data/tour-guide-data.csv')
guide_df

,Name,Gender,Domicile,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,...,CSN1,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5
0,Nabhan Nabilah,1,0,1,0,0,0,0,0,0,...,5,2,5,5,5,2,2,5,5,2
1,Salsabilla,1,0,1,0,0,0,0,0,1,...,4,3,3,4,4,2,2,2,4,2
2,Maulani,1,0,1,0,0,0,0,1,0,...,4,4,3,4,4,3,3,2,3,3
3,Abiyyu Farhan,0,0,1,0,0,0,0,0,1,...,4,5,3,5,4,4,4,4,4,4
4,Rahmaliyah Kadir,1,1,1,0,0,0,0,0,0,...,4,3,3,4,3,3,3,3,4,3
5,Faizah,1,1,1,0,0,0,0,0,0,...,4,2,5,5,5,3,4,5,5,4
6,Andani,1,1,0,1,0,0,0,0,1,...,4,4,4,3,3,4,3,3,3,3
7,Muthiah Hanun,1,1,0,1,0,0,0,0,0,...,5,5,5,5,5,5,4,3,5,4
8,Ihlasul Mufti Faqih,0,1,1,0,0,0,0,0,1,...,5,4,5,4,3,2,1,3,4,2
9,Dessylva Maharany Santosa,1,0,1,0,0,0,0,1,0,...,3,3,3,4,3,2,3,2,4,3


In [4]:
# Add roles column as a guide
guide_df['Roles'] = 'guide'
guide_df

,Name,Gender,Domicile,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,...,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5,Roles
0,Nabhan Nabilah,1,0,1,0,0,0,0,0,0,...,2,5,5,5,2,2,5,5,2,guide
1,Salsabilla,1,0,1,0,0,0,0,0,1,...,3,3,4,4,2,2,2,4,2,guide
2,Maulani,1,0,1,0,0,0,0,1,0,...,4,3,4,4,3,3,2,3,3,guide
3,Abiyyu Farhan,0,0,1,0,0,0,0,0,1,...,5,3,5,4,4,4,4,4,4,guide
4,Rahmaliyah Kadir,1,1,1,0,0,0,0,0,0,...,3,3,4,3,3,3,3,4,3,guide
5,Faizah,1,1,1,0,0,0,0,0,0,...,2,5,5,5,3,4,5,5,4,guide
6,Andani,1,1,0,1,0,0,0,0,1,...,4,4,3,3,4,3,3,3,3,guide
7,Muthiah Hanun,1,1,0,1,0,0,0,0,0,...,5,5,5,5,5,4,3,5,4,guide
8,Ihlasul Mufti Faqih,0,1,1,0,0,0,0,0,1,...,4,5,4,3,2,1,3,4,2,guide
9,Dessylva Maharany Santosa,1,0,1,0,0,0,0,1,0,...,3,3,4,3,2,3,2,4,3,guide


In [5]:
# Load Tourist Datasets
tourist_df = pd.read_csv('data/tourist-data.csv')
tourist_df

,Names,Genders,Destinations,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,...,CSN1,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5
0,Novia Rizki Wulandari,1,0,1,0,0,0,0,0,1,...,5,4,4,5,5,4,4,2,4,4
1,Herlina Kusyanuri Putri,1,0,1,0,0,0,0,1,0,...,5,5,5,5,5,4,3,3,4,3
2,Rifdah Alyaa,1,0,1,0,0,0,0,0,0,...,2,3,2,3,4,3,2,3,4,3
3,Alexis Purnomo,0,0,0,0,0,0,0,0,1,...,3,5,3,4,5,4,4,1,4,4
4,Rifky Surya Pratama,0,0,1,0,0,0,0,0,1,...,4,3,3,4,4,2,3,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Féi,1,1,1,0,0,0,0,0,0,...,4,2,4,4,4,3,4,2,4,4
69,Annisa,1,1,1,0,0,0,0,0,0,...,4,2,3,3,3,3,3,2,4,3
70,Usamah,0,1,1,0,0,0,0,0,1,...,5,5,4,5,5,4,4,5,5,5
71,Helena,1,0,0,0,0,0,0,1,0,...,4,3,4,4,4,3,4,2,5,4


In [6]:
# Add roles columns as a tourist
tourist_df['Roles'] = 'tourist'
tourist_df

,Names,Genders,Destinations,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,...,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5,Roles
0,Novia Rizki Wulandari,1,0,1,0,0,0,0,0,1,...,4,4,5,5,4,4,2,4,4,tourist
1,Herlina Kusyanuri Putri,1,0,1,0,0,0,0,1,0,...,5,5,5,5,4,3,3,4,3,tourist
2,Rifdah Alyaa,1,0,1,0,0,0,0,0,0,...,3,2,3,4,3,2,3,4,3,tourist
3,Alexis Purnomo,0,0,0,0,0,0,0,0,1,...,5,3,4,5,4,4,1,4,4,tourist
4,Rifky Surya Pratama,0,0,1,0,0,0,0,0,1,...,3,3,4,4,2,3,3,2,3,tourist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Féi,1,1,1,0,0,0,0,0,0,...,2,4,4,4,3,4,2,4,4,tourist
69,Annisa,1,1,1,0,0,0,0,0,0,...,2,3,3,3,3,3,2,4,3,tourist
70,Usamah,0,1,1,0,0,0,0,0,1,...,5,4,5,5,4,4,5,5,5,tourist
71,Helena,1,0,0,0,0,0,0,1,0,...,3,4,4,4,3,4,2,5,4,tourist


## Combine Datasets into Users Datasets

In [7]:
# Check if column names are the same
columns_equal = guide_df.columns.equals(tourist_df.columns)

if columns_equal:
    print("The DataFrames have the same column names.")
else:
    print("The DataFrames do not have the same column names.")

The DataFrames do not have the same column names.


In [8]:
# Get column names from guide_df and tourist_df
guide_columns = set(guide_df.columns)
tourist_columns = set(tourist_df.columns)

# Find the different columns
diff_col_in_guide = guide_columns - tourist_columns
diff_col_in_tourist = tourist_columns - guide_columns

# Print the different columns
print("Different columns in guide_df:", diff_col_in_guide)
print("Different columns in tourist_df:", diff_col_in_tourist)


Different columns in guide_df: {'Domicile', 'Name', 'Gender'}
Different columns in tourist_df: {'Destinations', 'Names', 'Genders'}


In [9]:
# Rename the diff column
guide_df = guide_df.rename(columns={'Name': 'Names'})
guide_df = guide_df.rename(columns={'Gender': 'Genders'})
guide_df = guide_df.rename(columns={'Domicile': 'Destinations'})


In [10]:
# Check if column names are the same
columns_equal = guide_df.columns.equals(tourist_df.columns)

if columns_equal:
    print("The DataFrames have the same column names.")
else:
    print("The DataFrames do not have the same column names.")

The DataFrames have the same column names.


In [11]:
user_df = pd.concat([tourist_df, guide_df])
user_df = user_df.reset_index(drop=True)
user_df

,Names,Genders,Destinations,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,...,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5,Roles
0,Novia Rizki Wulandari,1,0,1,0,0,0,0,0,1,...,4,4,5,5,4,4,2,4,4,tourist
1,Herlina Kusyanuri Putri,1,0,1,0,0,0,0,1,0,...,5,5,5,5,4,3,3,4,3,tourist
2,Rifdah Alyaa,1,0,1,0,0,0,0,0,0,...,3,2,3,4,3,2,3,4,3,tourist
3,Alexis Purnomo,0,0,0,0,0,0,0,0,1,...,5,3,4,5,4,4,1,4,4,tourist
4,Rifky Surya Pratama,0,0,1,0,0,0,0,0,1,...,3,3,4,4,2,3,3,2,3,tourist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Dera,1,0,1,0,0,0,0,0,0,...,4,5,5,5,4,5,5,5,5,guide
97,Salsabilla Rizki,1,0,1,0,0,0,0,0,1,...,5,5,5,5,5,5,2,3,4,guide
98,Salma,1,0,1,0,0,0,0,1,1,...,5,5,5,5,5,5,4,5,4,guide
99,Deva,1,0,0,1,0,0,0,0,1,...,4,4,3,4,3,4,2,4,3,guide


## Check Summary New Datasets

In [12]:
# Looking at the index of the columns
columns = user_df.columns
for idx, column in enumerate(columns):
    print(f"Column index: {idx}, Column name: {column}")

Column index: 0, Column name: Names
Column index: 1, Column name: Genders
Column index: 2, Column name: Destinations
Column index: 3, Column name: 17-25
Column index: 4, Column name: 26-34
Column index: 5, Column name: 35-43
Column index: 6, Column name: 44-52
Column index: 7, Column name: 52+
Column index: 8, Column name: Historical tours
Column index: 9, Column name: Adventure tours
Column index: 10, Column name: Nature and wildlife tours
Column index: 11, Column name: Culinary tours
Column index: 12, Column name: Wellness and retreat tours
Column index: 13, Column name: Architectural tours
Column index: 14, Column name: Educational tours
Column index: 15, Column name: Shopping tours
Column index: 16, Column name: EXT1
Column index: 17, Column name: EXT2
Column index: 18, Column name: EXT3
Column index: 19, Column name: EXT4
Column index: 20, Column name: EXT5
Column index: 21, Column name: EST1
Column index: 22, Column name: EST2
Column index: 23, Column name: EST3
Column index: 24,

In [13]:
user_df.describe()

,Genders,Destinations,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,Nature and wildlife tours,...,CSN1,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5
count,101.000000,101.000000,101.000000,101.000000,101.0,101.0,101.0,101.000000,101.000000,101.000000,...,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000
mean,0.702970,0.415842,0.702970,0.049505,0.0,0.0,0.0,0.386139,0.475248,0.653465,...,3.683168,3.396040,3.613861,3.811881,3.841584,3.297030,3.435644,2.910891,3.841584,3.445545
std,0.459229,0.495325,0.459229,0.218002,0.0,0.0,0.0,0.489291,0.501878,0.478239,...,1.009264,1.010734,1.009656,1.046068,0.945861,0.964827,0.973816,1.040183,0.977064,0.994739
min,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,3.000000,3.000000
50%,1.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,1.000000,...,4.000000,3.000000,4.000000,4.000000,4.000000,3.000000,3.000000,3.000000,4.000000,3.000000
75%,1.000000,1.000000,1.000000,0.000000,0.0,0.0,0.0,1.000000,1.000000,1.000000,...,4.000000,4.000000,4.000000,5.000000,5.000000,4.000000,4.000000,3.000000,5.000000,4.000000
max,1.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,1.000000,1.000000,1.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [14]:
# Check that we not missing any value after combine columns
print('Is there any missing value? ', user_df.isnull().values.any())
print('How many missing values? ', user_df.isnull().values.sum())

Is there any missing value?  False
How many missing values?  0


# **Concatenating Personality Type Prediction**

In [15]:
# Copy origin user data into match data to use in cluster
match_df = user_df.copy()

In [16]:
# Drop Column that not use for clustering match
match_df.drop(['Names', 'Destinations', 'Roles'], axis=1, inplace=True)
match_df

,Genders,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,Nature and wildlife tours,Culinary tours,...,CSN1,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5
0,1,1,0,0,0,0,0,1,1,0,...,5,4,4,5,5,4,4,2,4,4
1,1,1,0,0,0,0,1,0,0,1,...,5,5,5,5,5,4,3,3,4,3
2,1,1,0,0,0,0,0,0,1,1,...,2,3,2,3,4,3,2,3,4,3
3,0,0,0,0,0,0,0,1,1,0,...,3,5,3,4,5,4,4,1,4,4
4,0,1,0,0,0,0,0,1,1,1,...,4,3,3,4,4,2,3,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,1,0,0,0,0,0,0,1,1,...,5,4,5,5,5,4,5,5,5,5
97,1,1,0,0,0,0,0,1,1,1,...,3,5,5,5,5,5,5,2,3,4
98,1,1,0,0,0,0,1,1,0,1,...,5,5,5,5,5,5,5,4,5,4
99,1,0,1,0,0,0,0,1,1,1,...,4,4,4,3,4,3,4,2,4,3


In [17]:
persona_test_result = match_df.iloc[:, -25:]
persona_test_result

,EXT1,EXT2,EXT3,EXT4,EXT5,EST1,EST2,EST3,EST4,EST5,...,CSN1,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5
0,4,2,4,4,3,3,3,1,2,3,...,5,4,4,5,5,4,4,2,4,4
1,3,3,1,1,2,2,4,4,5,3,...,5,5,5,5,5,4,3,3,4,3
2,3,4,2,1,2,3,2,2,4,3,...,2,3,2,3,4,3,2,3,4,3
3,3,4,5,3,2,1,1,1,1,1,...,3,5,3,4,5,4,4,1,4,4
4,4,4,4,5,2,3,4,4,3,2,...,4,3,3,4,4,2,3,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5,4,5,4,5,4,3,3,5,4,...,5,4,5,5,5,4,5,5,5,5
97,3,4,3,3,3,2,4,2,2,2,...,3,5,5,5,5,5,5,2,3,4
98,2,1,4,4,4,4,5,5,2,1,...,5,5,5,5,5,5,5,4,5,4
99,4,4,4,4,4,2,3,2,4,3,...,4,4,4,3,4,3,4,2,4,3


In [18]:
# Convert input data to float32
data_input = np.array(persona_test_result, dtype=np.float32)

# Load model
model_path = 'persona_model/1'
loaded_model = tf.saved_model.load(model_path)

# Get the default signature of the model
inference_fn = loaded_model.signatures["serving_default"]

# Use models to make persona type predictions
predictions = inference_fn(tf.constant(data_input))["dense_2"]

In [19]:
predicted_clusters = np.argmax(predictions, axis=1)
persona_type = pd.DataFrame({'Persona type': predicted_clusters})
persona_type 

,Persona type
0,2
1,1
2,3
3,2
4,4
...,...
96,2
97,2
98,0
99,0


In [20]:
# Personality type column is changed to one hot encoding
persona_type = pd.get_dummies(persona_type, columns=['Persona type'])
persona_type = persona_type.astype(int)
persona_type

,Persona type_0,Persona type_1,Persona type_2,Persona type_3,Persona type_4
0,0,0,1,0,0
1,0,1,0,0,0
2,0,0,0,1,0
3,0,0,1,0,0
4,0,0,0,0,1
...,...,...,...,...,...
96,0,0,1,0,0
97,0,0,1,0,0
98,1,0,0,0,0
99,1,0,0,0,0


In [21]:
# Delete persona test result column in match_df
match_df = match_df.iloc[:, :-25]
match_df

,Genders,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,Nature and wildlife tours,Culinary tours,Wellness and retreat tours,Architectural tours,Educational tours,Shopping tours
0,1,1,0,0,0,0,0,1,1,0,0,0,0,1
1,1,1,0,0,0,0,1,0,0,1,0,1,0,0
2,1,1,0,0,0,0,0,0,1,1,0,1,0,0
3,0,0,0,0,0,0,0,1,1,0,0,1,0,0
4,0,1,0,0,0,0,0,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,1,0,0,0,0,0,0,1,1,0,0,0,1
97,1,1,0,0,0,0,0,1,1,1,0,0,0,0
98,1,1,0,0,0,0,1,1,0,1,0,1,0,0
99,1,0,1,0,0,0,0,1,1,1,0,0,0,0


In [22]:
# Concatenate 'match_df' with 'persona_type'
match_df = pd.concat([match_df, persona_type], axis=1)
match_df

,Genders,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,Nature and wildlife tours,Culinary tours,Wellness and retreat tours,Architectural tours,Educational tours,Shopping tours,Persona type_0,Persona type_1,Persona type_2,Persona type_3,Persona type_4
0,1,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0
1,1,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0
2,1,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0
97,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0
98,1,1,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0
99,1,0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0


# **Make PCA Model**

### Make PCA with Scaler Model

In [23]:
match_array = match_df.values
match_array

array([[1, 1, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 1, 0],
       ...,
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 0, 0]], dtype=int64)

In [24]:
class PCAModel(tf.Module):
    def __init__(self, x_mean, components):
        super(PCAModel, self).__init__()
        self.x_mean = tf.Variable(x_mean, trainable=False)
        self.components = tf.Variable(components, trainable=False)

    @tf.function(input_signature=[tf.TensorSpec(shape=(None, 19), dtype=tf.float32)])
    def scale_features(self, x):
        x_min = tf.reduce_min(x, axis=0)
        x_max = tf.reduce_max(x, axis=0)

        # Handle columns with zero variance
        zero_variance_mask = tf.math.equal(x_min, x_max)
        non_zero_variance_mask = tf.math.logical_not(zero_variance_mask)

        # Scale the columns with non-zero variance
        x_scaled_non_zero = tf.where(non_zero_variance_mask, (x - x_min) / (x_max - x_min), x)

        # Scale the columns with zero variance
        x_scaled_zero = tf.where(zero_variance_mask, tf.zeros_like(x), x_scaled_non_zero)

        return x_scaled_zero

    @tf.function(input_signature=[tf.TensorSpec(shape=(None, 19), dtype=tf.float32)])
    def apply_pca(self, x):
        x_scaled = self.scale_features(x)
        x_centered = x_scaled - self.x_mean
        
        # Compute the covariance matrix
        covariance_matrix = tf.matmul(tf.transpose(x_centered), x_centered) / tf.cast(tf.shape(x_centered)[0], dtype=tf.float32)

        # Perform eigenvalue decomposition
        eigenvalues, eigenvectors = tf.linalg.eigh(covariance_matrix)
        
        # Sort eigenvectors based on eigenvalues
        sorted_indices = tf.argsort(eigenvalues, direction='DESCENDING')
        sorted_eigenvectors = tf.gather(eigenvectors, sorted_indices, axis=1)
        
        # Select the top k eigenvectors
        k = tf.minimum(tf.shape(sorted_eigenvectors)[1], 2)  # Choose the top 2 eigenvectors (modify as needed)
        selected_eigenvectors = sorted_eigenvectors[:, :k]
        
        # Project the centered data onto the selected eigenvectors
        x_pca = tf.matmul(x_centered, selected_eigenvectors)
        
        return x_pca

# Convert the numpy array to a TensorFlow tensor
x_tf = tf.convert_to_tensor(match_array, dtype=tf.float32)

# Perform PCA with k=2 (reduce to 2 dimensions)
x_mean = tf.reduce_mean(x_tf, axis=0)
_, _, V = tf.linalg.svd(x_tf - x_mean)
components = V[:, :2]

# Create an instance of the PCA model
pca_model = PCAModel(x_mean, components)

In [25]:
# Save the PCAModel as a SavedModel
tf.saved_model.save(
    pca_model,
    export_dir='pca_model/1/',
    signatures={
        'serving_default': pca_model.apply_pca.get_concrete_function()
    }
)

INFO:tensorflow:Assets written to: pca_model/1/assets


In [26]:
!saved_model_cli show --dir {'pca_model/1/'} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['x'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 19)
        name: serving_default_x:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
The MetaGraph with tag set ['serve'] contains the following ops: {'TopKV2', 'AssignVariableOp', 'RealDiv', 'StringJoin', 'StatefulPartitionedCall', 'M

### Test Load Model Directly

In [27]:
# Load the PCA model
loaded_model = tf.saved_model.load("pca_model/1/")

In [28]:
# Generate a new data point
new_data = np.random.rand(1, 19)
new_data_tf = tf.convert_to_tensor(new_data, dtype=tf.float32)

# Predict the PCA value using the loaded model
pca_value = loaded_model.apply_pca(new_data_tf)

# Print the PCA value
print(pca_value)

tf.Tensor([[-1.6472878e+00  2.3469329e-07]], shape=(1, 2), dtype=float32)


In [30]:
# Predict PCA Valuse Using Our Match/Users Datasets
new_data_tf = tf.convert_to_tensor(match_array, dtype=tf.float32)

# Predict the PCA value using the loaded model
pca_value = loaded_model.apply_pca(new_data_tf)

# Print the PCA value
print(pca_value)

tf.Tensor(
[[ 0.90362394  0.4586686 ]
 [-1.1234949  -0.5327303 ]
 [-0.20840554  0.09403273]
 [-0.00210786  1.41256   ]
 [ 0.96277475  0.27177203]
 [ 0.06641878  0.49033108]
 [-0.646333    0.5166402 ]
 [ 0.80000377 -0.7733567 ]
 [-1.1317303   0.01277947]
 [ 0.1640983  -0.32443506]
 [-0.03162198  0.72825146]
 [-0.21996841  1.42571   ]
 [-0.8959171  -0.5676879 ]
 [-0.68994904 -0.5651168 ]
 [-0.56146425  0.03711636]
 [-1.0953424   0.23444465]
 [ 0.33884782  0.09858982]
 [ 0.00608534 -0.35555592]
 [-0.51399726  0.15410763]
 [-0.6629581   0.41797698]
 [-0.7578075  -0.7598223 ]
 [-0.49729478 -0.43017465]
 [-1.0476477   0.31849688]
 [ 0.9739295  -0.16895357]
 [ 0.5684941  -0.5919983 ]
 [-1.3624796  -0.00943238]
 [ 0.08562335  1.3656352 ]
 [-0.5247066   0.07196948]
 [ 0.09501052 -0.08412665]
 [-0.32602304 -0.86611736]
 [-0.53399134  0.19448036]
 [ 0.8953172   0.06471789]
 [-0.00255138  0.15033323]
 [ 0.6424954  -0.7136053 ]
 [ 0.46385843 -0.40493926]
 [-1.4939647   0.09693141]
 [-0.17456031  0.